# Activity: Fun with 32-bit Floating Point Numbers
Most of the time (depending on your hardware), you'll use 64-bit floating-point numbers by default. However, many machine learning applications use `Float32` as their default precision. 
* _Why_? Float32 offers a sweet spot for deep learning: its 24‐bit significand (which gives seven decimal digits) and 8‐bit exponent provide sufficient precision and dynamic range for most applications, halving the memory footprint compared to Float64. This enables computation leveraging specialized hardware optimized specifically for 32-bit arithmetic.

In this activity, you'll explore the memory layout of `Float32`, and compute this floating point type's dynamic range and precision limits.

## Example memory layout
Suppose we have a floating point number $x\in\mathbb{R}$ that is approximated as a 32-bit variable in memory. A 32-bit number $x\in\mathbb{R}$ is encoded in memory as:
$$
\begin{align*}
x = \underbrace{S}_{\text{sign}}\times\underbrace{\text{significand}}_{\text{fraction}}\times\underbrace{{2^{E-127}}}_{\text{scale}}
\end{align*}
$$
where:
$$
\begin{align*}
S &= -1^{d_{31}}\\
\text{significand} &= 1 + \sum_{i = 1}^{23}d_{i}2^{-i}\\
E &= \sum_{i=23}^{30}d_{i}2^{i - 23}
\end{align*}
$$
where $d_{i}$ denotes the digit at position $i$ in the number. Notice the difference between the 64- and 32-bit numbers: the number of elements used to compute the significand and the exponent terms are different, and the location of the sign bit has changed, but otherwise they have a similar structural layout in memory.

Now, let's compute the components of the 32-bit representation of $x\in\mathbb{R}$. First, specify an example number, save it in the `x::Float32` variable:

In [9]:
x = 141.72 |> Float32; # why do we need |> Float32?

Check the type using [the `typeof(...)` method](https://docs.julialang.org/en/v1/base/base/#Core.typeof):

In [6]:
typeof(x) == Float32 # if Float32, this should be true

true

Next, let's use [the `bitstring(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring) to generate the bits of our 32-bit floating point number $x$ as a `String`, and then convert and save the bitstring into a `0`-based dictionary called `d::Dict{Int,Int}`:

In [34]:
d = let

    # initialize -
    bitpattern_dictionary = Dict{Int64,Int64}(); # storage for the 0-based bit pattern
    wordsize = 32; # how many boxes do we have?
    a = bitstring(x) |> reverse |> collect .|> v-> parse(Int64, v) # fancy. Nothing to see here, move along (for now anyway).
    
    # put stuff in the dictionary
    for i ∈ 0:(wordsize-1)
        bitpattern_dictionary[i] = a[i+1];
    end
    bitpattern_dictionary # return to caller
end;

Now that we have the bitpattern dictionary `d::Dict{Int, Int}`, we can compute the three components of our floating point number. Let's start with the sign, which we'll save in the `S:Float64` variable:

In [27]:
S = let
    S = (-1.0)^(d[31]);
end

1.0

Next, let's compute a value for the `significand` of $x\in\mathbb{x}$, which we'll store in the `calculated_significand_value::Float64` variable:

In [64]:
calculated_significand_value = let

    # initialize -
    calculated_significand_value = 0.0;
    b = 2.0; # binary, base = 2
    msb = 23; # most significant bit (msb)
    lsb = 1; # least significant bit (lsb)
    significand_range_array = range(lsb,stop=msb,step=1) |> collect; # range of bits to use for the significand

    for i ∈ significand_range_array
        calculated_significand_value += (b^(-i))*d[msb-i]
    end
    calculated_significand_value + 1
end

1.1071875095367432

_Check_: Let's use [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check our calculated significand value. If the `==` comparision comes back `false`, [an `AssertionError` is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError):

In [66]:
@assert significand(x) == calculated_significand_value # compare built-in versus our calculated value

Finally, let's compute the scale of the floating point number $x\in\mathbb{R}$, which requires that we compute the $E$ value (which we'll store in the `E::Float64` variable). 

In [72]:
E = let

    # initialize -
    calculated_exponent_value = 0.0;
    b = 2.0; # binary, base = 2
    msb = 30; # most significant bit (msb)
    lsb = 23; # least significant bit (lsb)
    exponent_bit_range_array = range(lsb, stop=msb, step = 1) |> collect

    for i ∈ eachindex(exponent_bit_range_array)
        j = exponent_bit_range_array[i]; # remap operation
        calculated_exponent_value += d[j]*(b^(i))
    end
    calculated_exponent_value
end

268.0

In [70]:
2^(1.04 - 127)

1.208541995236496e-38